In [1]:
import sqlite3
import pandas as pd
import numpy as np

import os
import sys
from datetime import timedelta, date
import csv

# Alexa with OpenINTEL data

In [2]:
db = sqlite3.connect('[...PATH...]/data/openintel-alexa1m/processed/dns_with_alexa-HYBRID.db')

In [3]:
df = pd.read_sql_query("SELECT * from dns_alexa_v4", db)
df

,rank,query_name,cdn_name,cdn,date
0,1,www.google.com.,Google,1.0,2019-12-01
1,2,www.youtube.com.,Google,1.0,2019-12-01
2,3,www.tmall.com.,Taobao,1.0,2019-12-01
3,4,www.baidu.com.,None,0.0,2019-12-01
4,5,www.amazon.com.,Amazon CloudFront,1.0,2019-12-01
...,...,...,...,...,...
22740731,537429,www.jp.life.,None,NaN,None
22740732,537430,www.kampiun.id.,None,0.0,2019-12-31
22740733,537431,www.myhlc-charlottesville.com.,None,0.0,2019-12-31
22740734,537432,www.not3.io.,Amazon,1.0,2019-12-31


In [4]:
df_6 = pd.read_sql_query("SELECT * from dns_alexa_v6", db)
df_6

,rank,query_name,cdn_name,cdn,date
0,1,www.google.com.,Google,1.0,2019-12-01
1,2,www.youtube.com.,Google,1.0,2019-12-01
2,3,www.tmall.com.,None,NaN,None
3,4,www.baidu.com.,None,NaN,None
4,5,www.amazon.com.,None,NaN,None
...,...,...,...,...,...
22736874,537429,www.jp.life.,None,NaN,None
22736875,537430,www.kampiun.id.,None,NaN,None
22736876,537431,www.myhlc-charlottesville.com.,None,NaN,None
22736877,537432,www.not3.io.,None,NaN,None


In [5]:
df['cdn']=df['cdn'].fillna(0)
df_6['cdn']=df_6['cdn'].fillna(0)
df_6['cdn_name']=df_6['cdn_name'].fillna('None')
df['cdn_name']=df['cdn_name'].fillna('None')

In [7]:
df['cdn_name'].replace({'Amazon CloudFront' : 'Amazon', 'Microsoft Azure' : 'Microsoft'}, inplace=True)
df_6['cdn_name'].replace({'Amazon CloudFront' : 'Amazon', 'Microsoft Azure' : 'Microsoft'}, inplace=True)

In [8]:
df['cdn_name'].unique()

array(['Google', 'Taobao', 'None', 'Amazon', 'Akamai', 'Facebook',
       'Yahoo', 'Fastly', 'Microsoft', 'Twitter', 'Cloudflare',
       'Reflected Networks', 'WordPress', 'ChinaCache', 'Cedexis',
       'Edgecast', 'Azion', 'GoCache', 'ChinaNetCenter', 'Incapsula',
       'StackPath', 'LeaseWeb CDN', 'Highwinds', 'SFR', 'Medianova',
       'Instart Logic', 'Limelight', 'CDNetworks', 'Level3', 'NetDNA',
       'Yottaa', 'Rackspace', 'AT&T', 'BitGravity', 'Internap', 'Level 3',
       'NGENIX', 'KeyCDN', 'CDNvideo', 'Netlify', 'Zenedge', 'Telenor',
       'CDN77', 'Singular CDN', 'SwiftCDN', 'KINX CDN', 'XLabs Security',
       'Optimal CDN', 'NYI FTW', 'OnApp', 'BunnyCDN', 'BelugaCDN',
       'cubeCDN', 'Aryaka', 'Ananke', 'PUSHR', 'TRBCDN', 'Cachefly',
       'CacheFly', 'Roast.io', 'Bison Grid', 'Universal CDN', 'BO.LT',
       'PageCDN'], dtype=object)

In [9]:
df_6['cdn_name'].unique()

array(['Google', 'None', 'Akamai', 'Facebook', 'Yahoo', 'Microsoft',
       'Amazon', 'Cedexis', 'Cloudflare', 'Fastly', 'ChinaNetCenter',
       'LeaseWeb CDN', 'Incapsula', 'Instart Logic', 'KeyCDN', 'Edgecast',
       'ChinaCache', 'Rackspace', 'Netlify', 'BitGravity', 'OnApp',
       'Limelight', 'Level3', 'Telenor', 'AT&T', 'BelugaCDN', 'Internap',
       'Level 3', 'BunnyCDN', 'CDN77', 'CDNetworks', 'WordPress',
       'Yottaa', 'Highwinds'], dtype=object)

# CDN Penetration per CDN

In [10]:
df_ranked = df[['rank', 'cdn', 'date', 'cdn_name']]

In [11]:
df_cdn=df.groupby(['cdn_name','date']).agg({'query_name':pd.Series.nunique}).reset_index()
df_cdn['date']=pd.to_datetime(df_cdn['date'])
df_cdn=df_cdn.set_index('date')

df_cdn_list = df_cdn[df_cdn['cdn_name']!='None'][['cdn_name','query_name']].groupby(["cdn_name"]).median().sort_values("query_name",ascending=False)
df_cdn_list = df_cdn_list[df_cdn_list.query_name >=10]
cdn_list = df_cdn_list.reset_index().iloc[:,0].tolist()[:10]


x1 = np.logspace(0.0, 6.0, 7, True)
x2 = np.linspace(0,1000000,101, True)
x2 = np.union1d(x1,x2)
x2 = x2.tolist()
del x2[0]

df_plt3 = pd.DataFrame(columns=['top_max_rank', 'query_type', 'cdn_penetration', 'cdn_name', 'type'])
i = 0
for alexa_index in x2:
    if alexa_index == 0:
        alexa_index += 1
    df_tmp = df_ranked[(df_ranked['rank']>=1) & (df_ranked['rank']<=alexa_index)]
    df_tmp_cdn = df_tmp.groupby(['cdn_name','date']).agg({'cdn':'sum'}).reset_index()
    df_tmp_all = df_tmp.groupby(['date']).agg({'cdn':'sum'})
    df_tmp_cdn_pen_merged = pd.merge(df_tmp_cdn, df_tmp_all.reset_index(), how='left', on=['date'])
    df_tmp_cdn_pen_merged['cdn_pen'] = df_tmp_cdn_pen_merged['cdn_x'] / df_tmp_cdn_pen_merged['cdn_y']
    
    df_plt3.loc[i] = [alexa_index, 'A', (df_tmp_all.median() / alexa_index)[0], 'All', 'All']
    
    n=1
    for cdn in cdn_list:
        df_plt3.loc[i+n] = [alexa_index, 'A', df_tmp_cdn_pen_merged[df_tmp_cdn_pen_merged.cdn_name == cdn]['cdn_pen'].median(), cdn, 'CDN div all']
        df_plt3.loc[i+n+1] = [alexa_index, 'A', (df_tmp_cdn[df_tmp_cdn.cdn_name == cdn].median() / alexa_index)[0], cdn, 'CDN div rank']
        n += 2
    
    
    i+= 21
    
df_plt3

,top_max_rank,query_type,cdn_penetration,cdn_name,type
0,1.0,A,1.000000,All,All
1,1.0,A,NaN,Cloudflare,CDN div all
2,1.0,A,NaN,Cloudflare,CDN div rank
3,1.0,A,NaN,Amazon,CDN div all
4,1.0,A,NaN,Amazon,CDN div rank
...,...,...,...,...,...
2179,1000000.0,A,0.003430,LeaseWeb CDN,CDN div rank
2180,1000000.0,A,0.011993,Incapsula,CDN div all
2181,1000000.0,A,0.002831,Incapsula,CDN div rank
2182,1000000.0,A,0.009148,Rackspace,CDN div all


In [12]:
df_plt3['cdn_penetration'] = df_plt3['cdn_penetration'].fillna(0)

In [13]:
db_for_saving_this = sqlite3.connect('[...PATH...]/data/openintel-alexa1m/processed/dns_with_alexa_cdn_pen-HYBRID.db')

df_plt3.to_sql(name='df_cdn_penetration_v4', con=db_for_saving_this, if_exists='append', index=False)






# CDN Penetration per CDN for IPv6

In [14]:
df_ranked = df_6[['rank', 'cdn', 'date', 'cdn_name']]

In [15]:
df_cdn=df_6.groupby(['cdn_name','date']).agg({'query_name':pd.Series.nunique}).reset_index()
df_cdn['date']=pd.to_datetime(df_cdn['date'])
df_cdn=df_cdn.set_index('date')

df_cdn_list = df_cdn[(df_cdn['cdn_name']!='None') & (df_cdn['cdn_name']!='No IPv6')][['cdn_name','query_name']].groupby(["cdn_name"]).median().sort_values("query_name",ascending=False)
df_cdn_list = df_cdn_list[df_cdn_list.query_name >=10]
cdn_list = df_cdn_list.reset_index().iloc[:,0].tolist()[:10]


x1 = np.logspace(0.0, 6.0, 7, True)
x2 = np.linspace(0,1000000,101, True)
x2 = np.union1d(x1,x2)
x2 = x2.tolist()
del x2[0]

df_plt4 = pd.DataFrame(columns=['top_max_rank', 'query_type', 'cdn_penetration', 'cdn_name', 'type'])
i = 0
for alexa_index in x2:
    if alexa_index == 0:
        alexa_index += 1
    df_tmp = df_ranked[(df_ranked['rank']>=1) & (df_ranked['rank']<=alexa_index)]
    df_tmp_cdn = df_tmp.groupby(['cdn_name','date']).agg({'cdn':'sum'}).reset_index()
    df_tmp_all = df_tmp.groupby(['date']).agg({'cdn':'sum'})
    df_tmp_cdn_pen_merged = pd.merge(df_tmp_cdn, df_tmp_all.reset_index(), how='left', on=['date'])
    df_tmp_cdn_pen_merged['cdn_pen'] = df_tmp_cdn_pen_merged['cdn_x'] / df_tmp_cdn_pen_merged['cdn_y']
    
    df_plt4.loc[i] = [alexa_index, 'AAAA', (df_tmp_all.median() / alexa_index)[0], 'All', 'All']
    
    n=1
    for cdn in cdn_list:
        df_plt4.loc[i+n] = [alexa_index, 'AAAA', df_tmp_cdn_pen_merged[df_tmp_cdn_pen_merged.cdn_name == cdn]['cdn_pen'].median(), cdn, 'CDN div all']
        df_plt4.loc[i+n+1] = [alexa_index, 'AAAA', (df_tmp_cdn[df_tmp_cdn.cdn_name == cdn].median() / alexa_index)[0], cdn, 'CDN div rank']
        n += 2
    
    
    i+= 21
    
df_plt4['cdn_penetration'] = df_plt4['cdn_penetration'].fillna(0)
df_plt4

,top_max_rank,query_type,cdn_penetration,cdn_name,type
0,1.0,AAAA,1.000000,All,All
1,1.0,AAAA,0.000000,Cloudflare,CDN div all
2,1.0,AAAA,0.000000,Cloudflare,CDN div rank
3,1.0,AAAA,1.000000,Google,CDN div all
4,1.0,AAAA,1.000000,Google,CDN div rank
...,...,...,...,...,...
2179,1000000.0,AAAA,0.000129,Incapsula,CDN div rank
2180,1000000.0,AAAA,0.000424,LeaseWeb CDN,CDN div all
2181,1000000.0,AAAA,0.000054,LeaseWeb CDN,CDN div rank
2182,1000000.0,AAAA,0.000265,Rackspace,CDN div all


In [16]:
df_plt4

,top_max_rank,query_type,cdn_penetration,cdn_name,type
0,1.0,AAAA,1.000000,All,All
1,1.0,AAAA,0.000000,Cloudflare,CDN div all
2,1.0,AAAA,0.000000,Cloudflare,CDN div rank
3,1.0,AAAA,1.000000,Google,CDN div all
4,1.0,AAAA,1.000000,Google,CDN div rank
...,...,...,...,...,...
2179,1000000.0,AAAA,0.000129,Incapsula,CDN div rank
2180,1000000.0,AAAA,0.000424,LeaseWeb CDN,CDN div all
2181,1000000.0,AAAA,0.000054,LeaseWeb CDN,CDN div rank
2182,1000000.0,AAAA,0.000265,Rackspace,CDN div all


In [17]:
df_plt4.to_sql(name='df_cdn_penetration_v6', con=db_for_saving_this, if_exists='append', index=False)



# Look into why Google increases towards the tail of the CDN Penetration plot

In [18]:
df_ranked = df[['rank', 'cdn', 'date', 'cdn_name', 'query_name']]
df_ranked_6 = df_6[['rank', 'cdn', 'date', 'cdn_name', 'query_name']]

In [ ]:
x1 = np.logspace(0.0, 6.0, 7, True)
x2 = np.linspace(0,1000000,101, True)
x2 = np.union1d(x1,x2)
x2 = x2.tolist()
del x2[0]

df_plt_google = pd.DataFrame(columns=['top_max_rank', 'query_type', 'blogspot_penetration'])
i = 0
for alexa_index in x2:
    if alexa_index == 0:
        alexa_index += 1
        
    
    df_tmp_4 = df_ranked[(df_ranked['rank']>=1) & (df_ranked['rank']<=alexa_index) & (df_ranked['cdn_name'] == 'Google')]
    df_tmp_6 = df_ranked_6[(df_ranked_6['rank']>=1) & (df_ranked_6['rank']<=alexa_index) & (df_ranked['cdn_name'] == 'Google')]
    df_tmp_cdn_4 = df_ranked[(df_ranked['rank']>=1) & (df_ranked['rank']<=alexa_index)]
    df_tmp_cdn_6 = df_ranked_6[(df_ranked_6['rank']>=1) & (df_ranked_6['rank']<=alexa_index)]
    
    df_tmp_cdn_pen_merged_4 = pd.merge(df_tmp_4[df_tmp_4.query_name.str.contains('\.blogspot\.')].groupby(['date']).agg({'cdn':'sum'}), df_tmp_cdn_4.groupby(['date']).agg({'cdn':'sum'}), how='left', on=['date'])
    df_tmp_cdn_pen_merged_4['cdn_pen'] = df_tmp_cdn_pen_merged_4['cdn_x'] / df_tmp_cdn_pen_merged_4['cdn_y']
    
    df_tmp_cdn_pen_merged_6 = pd.merge(df_tmp_6[df_tmp_6.query_name.str.contains('\.blogspot\.')].groupby(['date']).agg({'cdn':'sum'}), df_tmp_6.groupby(['date']).agg({'cdn':'sum'}), how='left', on=['date'])
    df_tmp_cdn_pen_merged_6['cdn_pen'] = df_tmp_cdn_pen_merged_6['cdn_x'] / df_tmp_cdn_pen_merged_6['cdn_y']
    
    
                                       

                                       
    df_plt_google.loc[i] = [alexa_index, 'A', df_tmp_cdn_pen_merged_4['cdn_pen'].median()]
    df_plt_google.loc[i+1] = [alexa_index, 'AAAA', df_tmp_cdn_pen_merged_4['cdn_pen'].median()]
    
                                       
    
    
    i+= 2
    
df_plt_google



In [20]:

df_plt_google['blogspot_penetration'] = df_plt_google['blogspot_penetration'].fillna(0)


In [21]:
df_plt_google.to_sql(name='df_cdn_penetration_blogspot', con=db_for_saving_this, if_exists='append', index=False)


In [22]:
db.close()
db_for_saving_this.close()

In [26]:
df_plt3.head()

,top_max_rank,query_type,cdn_penetration,cdn_name,type
0,1.0,A,1.0,All,All
1,1.0,A,0.0,Cloudflare,CDN div all
2,1.0,A,0.0,Cloudflare,CDN div rank
3,1.0,A,0.0,Amazon,CDN div all
4,1.0,A,0.0,Amazon,CDN div rank


In [28]:
df_plt4.head()

,top_max_rank,query_type,cdn_penetration,cdn_name,type
0,1.0,AAAA,1.0,All,All
1,1.0,AAAA,0.0,Cloudflare,CDN div all
2,1.0,AAAA,0.0,Cloudflare,CDN div rank
3,1.0,AAAA,1.0,Google,CDN div all
4,1.0,AAAA,1.0,Google,CDN div rank


In [27]:
df_plt_google.head()

,top_max_rank,query_type,blogspot_penetration
0,1.0,A,0.000000
1,1.0,AAAA,0.000000
2,10.0,A,0.000000
3,10.0,AAAA,0.000000
4,100.0,A,0.016667
